In [1]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import layers,Input
from keras.losses import SparseCategoricalCrossentropy,BinaryCrossentropy
import numpy as np
import os
from tensorflow.python.client import device_lib
from sklearn.metrics import accuracy_score

2024-04-05 18:39:48.125439: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-05 18:39:48.125562: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-05 18:39:48.280116: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
train1=os.listdir('/kaggle/input/yolo-audi')
train2=os.listdir('/kaggle/input/yolo-swift')

In [4]:
y=[]

In [5]:
train_path = '/kaggle/input/yolo-audi/'
data = []
for img_file in train1:
    image = Image.open(train_path + img_file)
    image = image.resize((448, 448))
    image = image.convert('RGB')
    image = np.array(image)
    data.append(image)
for img_file in train1:
    y.append(1)
train_path = '/kaggle/input/yolo-swift/'
for img_file in train2:
    image = Image.open(train_path + img_file)
    image = image.resize((448,448))
    image = image.convert('RGB')
    image = np.array(image)
    data.append(image)
for img_file in train2:
    y.append(0)

In [6]:
x=np.array(data)
y=np.array(y)

In [7]:
y.shape

(400,)

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [9]:
# scaling the data
x_train_scaled = x_train/255
x_test_scaled = x_test/255

In [10]:
print(x_test_scaled.shape)

(120, 448, 448, 3)


In [11]:
input_shape=(448,448,3)
model = tf.keras.Sequential([


    #layer1
    Input(shape=input_shape),
    layers.Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), padding='same', activation='relu'),#input_shape=input_shape 
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    
    #layer 2
    layers.Conv2D(filters=192,kernel_size=(3, 3), padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),

    # Layer 3.1
    layers.Conv2D(filters=128,kernel_size= (1, 1), padding='same', activation='relu'),
    layers.Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu'),
    # Layer 3.2
    layers.Conv2D(filters=256, kernel_size=(1, 1), padding='same', activation='relu'),
    layers.Conv2D(filters=512, kernel_size=(3, 3), padding='same', activation='relu'),#changed here<----------1
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),


    # Layer 5
    layers.Conv2D(filters=256,kernel_size= (1, 1), padding='same', activation='relu'),
    layers.Conv2D(filters=512,kernel_size= (3, 3), padding='same', activation='relu'),
    layers.Conv2D(filters=256,kernel_size= (1, 1), padding='same', activation='relu'),
    layers.Conv2D(filters=512,kernel_size= (3, 3), padding='same', activation='relu'),

    layers.Conv2D(filters=256,kernel_size= (1, 1), padding='same', activation='relu'),#changed here<----------3
    layers.Conv2D(filters=512,kernel_size= (3, 3), padding='same', activation='relu'),

    layers.Conv2D(filters=256,kernel_size= (1, 1), padding='same', activation='relu'),
    layers.Conv2D(filters=512,kernel_size= (3, 3), padding='same', activation='relu'),#changed here<----------4

    layers.Conv2D(filters=512,kernel_size= (1, 1), padding='same', activation='relu'),
    layers.Conv2D(filters=1024,kernel_size= (3, 3), padding='same', activation='relu'),#changed here<----------7
    layers.BatchNormalization(),
    layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),


    #layer 6
    layers.Conv2D(filters=512,kernel_size=(1,1),padding="same",activation="relu"),
    layers.Conv2D(filters=1024,kernel_size=(3,3),padding="same",activation="relu"),

    layers.Conv2D(filters=512,kernel_size=(1,1),padding="same",activation="relu"),#changed here<----------5
    layers.Conv2D(filters=1024,kernel_size=(3,3),padding="same",activation="relu"),

    layers.Conv2D(filters=512,kernel_size=(1,1),padding="same",activation="relu"),
    layers.Conv2D(filters=1024,kernel_size=(3,3),padding="same",activation="relu"),#changed here<----------2

    layers.Conv2D(filters=512,kernel_size=(1,1),padding="same",activation="relu"),
    layers.Conv2D(filters=1024,kernel_size=(3,3),padding="same",activation="relu"),


    #layer 7
    layers.Conv2D(filters=1024,kernel_size=(3,3),padding="same",activation="relu"),
    layers.Conv2D(filters=1024,kernel_size=(3,3),padding="same",activation="relu"),#changed here<----------6

    #layer 8
    layers.Flatten(),
    layers.Dense(2300, activation='relu'),
    layers.Dense(1, activation='sigmoid')

])

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 224, 224, 64)   │         9,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 112, 112, 192)  │       110,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 112, 112, 192)  │           768 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 192)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 56, 56, 128)    │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 56, 56, 256)    │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 56, 56, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 56, 56, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 28, 28, 256)    │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 256)    │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 28, 28, 256)    │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 28, 28, 256)    │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 28, 28, 512)    │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 28, 28, 1024)   │     4,719,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 28, 28, 1024)   │         4,096 │
│ (BatchNormalization)            │                        │             

 Total params: 513,399,353 (1.91 GB)

 Trainable params: 513,395,769 (1.91 GB)

 Non-trainable params: 3,584 (14.00 KB)

In [12]:
model.fit(x_train_scaled, y_train,epochs=50)

Epoch 1/50


W0000 00:00:1712342517.017611      84 hlo_rematerialization.cc:2946] Can't reduce memory use below 6.56GiB (7044888635 bytes) by rematerialization; only reduced to 11.70GiB (12559341464 bytes), down from 12.92GiB (13871954904 bytes) originally
I0000 00:00:1712342521.987768      84 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1712342522.023132      84 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


8/9 ━━━━━━━━━━━━━━━━━━━━ 0s 372ms/step - accuracy: 0.5043 - loss: 153.9325

W0000 00:00:1712342559.858593      83 hlo_rematerialization.cc:2946] Can't reduce memory use below 6.58GiB (7063192901 bytes) by rematerialization; only reduced to 11.19GiB (12019733528 bytes), down from 12.16GiB (13055961096 bytes) originally
W0000 00:00:1712342565.490686      83 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


9/9 ━━━━━━━━━━━━━━━━━━━━ 103s 5s/step - accuracy: 0.5099 - loss: 139.1577
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 365ms/step - accuracy: 0.5592 - loss: 0.8522
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 365ms/step - accuracy: 0.6166 - loss: 1.1221
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 365ms/step - accuracy: 0.5451 - loss: 0.7628
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 365ms/step - accuracy: 0.5008 - loss: 0.7909
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 364ms/step - accuracy: 0.5456 - loss: 0.6660
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 366ms/step - accuracy: 0.6587 - loss: 0.6448
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 365ms/step - accuracy: 0.5755 - loss: 0.6440
Epoch 9/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 365ms/step - accuracy: 0.6249 - loss: 0.6541
Epoch 10/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 365ms/step - accuracy: 0.6445 - loss: 0.6231
Epoch 11/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 365ms/step - accuracy: 0.7054 - loss: 0.5760
Epoch 12/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 365ms/step - accuracy: 0.6883 - loss: 0.6351

In [63]:
y_pred=model.predict(x_test_scaled)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step


In [117]:
for i in range(0,5):
    if(y_pred[i]>0.40):
        y_pred[i]=1.0
    else:
        y_pred[i]=0.0

In [118]:
y_test[:5]

array([1, 1, 1, 0, 0])

In [119]:
y_pred[:5].flatten()

array([1., 1., 0., 0., 0.], dtype=float32)